In [1]:
import os
import openpyxl

In [2]:
path_list = []

for curDir, dirs, files in os.walk("新聞"):
    #         print('=='*20)
    #         print('現在のフォルダ: ' + curDir)
    #         print('内包するフォルダ: ', end='')
    #         print(dirs)
    #         print('内包するファイル: ', end='')
    #         print(files)
    path = curDir + '\\'
    for file in files:
        path += file
        #             print('ファイルパス: ' + path)
        path_list.append(path)
        path = curDir + '\\'

In [3]:
headlines = []  # 格納用辞書

wb1 = openpyxl.load_workbook(path_list[0])
sheet = wb1['シート1']
# [社名、年、月、名前、見出し、ベクトル]
headlines = [(row[0].value, row[1].value, row[2].value, row[3].value, row[4].value) for row in sheet.rows]

wb2 = openpyxl.load_workbook(path_list[1])
sheet = wb2['シート1']
headlines2 = [(row[0].value, row[1].value, row[2].value, row[3].value, row[4].value) for row in sheet.rows]

In [4]:
headlines += headlines2

# 分かち書き

In [5]:
import MeCab as mc
import mojimoji
m = mc.Tagger("-Ochasen")

In [12]:
def mecab_tokenizer(text: str):
    """テキストを分かち書き

    Args:
        text (str): 分割したいテキスト

    Returns:
        [List(str,str,...)]: 分割後のテキスト
    """

    node = m.parseToNode(text)
    word_list = list()
    while node:
        if node.surface != "":
            res = node.feature.split(",")
            word_type = res[0]
            if word_type in ['名詞', "動詞", "形容詞", "副詞"]:  # 名詞, 動詞, 形容詞, 副詞のみを抽出
                basic_word = res[6]
                if basic_word != "*":
                    word_list.append(basic_word)
                else:
                    word_list.append('[UNK]')  # 未知語の場合は[UNK]トークンに置き換え
        node = node.next
        if node is None:
            break
    return word_list

In [13]:
def clean_text(text: str):
    """[summary]

    Args:
        text (str): 正規化するテキスト

    Returns:
        str: 正規化されたテキスト
    """

    text = mojimoji.han_to_zen(text, digit=False, ascii=False)  # 半角文字を全角文字に統一(数字, 英語以外)
    text = mojimoji.zen_to_han(text, kana=False)  # 全角文字を半角文字に統一(かな以外)
    text = text.lower()  # 小文字に統一
    return text

# ベクトル化

In [7]:
from gensim.models import KeyedVectors

In [9]:
import numpy as np

In [10]:
file_name = "jawiki.all_vectors.200d-002.txt"

model = KeyedVectors.load_word2vec_format(file_name)

In [11]:
def vectorize_word(text: str):
    """word2vecを用いたテキストのベクトル化

    元word_analysis()

    Args:
        text (str): ベクトル化するテキスト

    Returns:
        numpy.ndarray: テキストのベクトル
    """

    text = clean_text(text)

    V = list()  # 文章のベクトル(200次元)を格納

    # 文章の対して, 文章中の単語のベクトルの平均を求める処理を行う
    word_list = mecab_tokenizer(text)
    v = np.array([0.0] * 200)
    word_num = 0
    for word in word_list:
        try:
            v += np.array(model[word])
        except KeyError as error:
            continue
        except ValueError as verror:
            continue
        word_num += 1

    try:
        v = v / word_num
    except ZeroDivisionError as e:
        print(f'ZeroDivisionError: {e}')

    return v

def cos_sim(v1, v2):
    """2つのベクトルのcos類似度を計算

    Args:
        v1 (numpy.ndarray): word2vecによりベクトル化したテキスト1
        v2 (numpy.ndarray): word2vecによりベクトル化したテキスト2

    Returns:
        [float]: 二つのテキストのcos類似度
    """
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [14]:
def get_news_dict(filename: str):
    """テキストのparseとベクトル化

    エクセルから見出しのタイトル(識別番号？)とテキストを取り出しベクトル化して辞書として返す

    Args:
        similarity (SimilarityByWord2vec): sim_by_w2vオブジェクト<br>ベクトル化に使用
        filename (str): excelファイル名

    Returns:
        {(str, str, str, str): (str, numpy.ndarray)}: ひと月分の辞書<br>一つの要素に{(会社名、年、月、名前):(見出し, ベクトル)} -> の形で返す
    """

    wb = openpyxl.load_workbook(filename)
    sheet = wb['シート1']
    dict = {}  # 格納用辞書
    for row in sheet.rows:

        if row[4].value is not None:
            vector = vectorize_word(row[4].value)  # 見出しをベクトル化
            dict[(row[0].value, row[1].value, row[2].value, row[3].value)] = (row[4].value, vector)  # (会社名、年、月、名前)と(見出し、ベクトル)の組

    return dict

In [15]:
all_newspapers = [get_news_dict(path) for path in path_list]

headline_list = [(name[0], name[1], name[2], name[3], headline[0], headline[1]) for news in all_newspapers \
                for name, headline in news.items()]

del all_newspapers

C:\Users\kenbun\anaconda3\envs\nlp_bot\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide


In [16]:
def get_all_similarity(headline_list, speech_text: str):
    """全見出しと発言の類似度分析

    全ての見出しと発言の類似度分析を行い, 類似度で降順にソートして返す

    Args:
        headline_list (List(str, str, tuple(str, numpy.ndarray))): 見出しをまとめたリスト<br> 
        ひとつの要素に(ファイルパス, 番号, (見出し, ベクトル)) -> の形で格納している
        speech_text (str): 類似度分析を行うテキスト

    Returns:
        List(tuple(tuple(str, str, str, str, str), float)): 類似度で降順にソートしたリスト [((会社名, 出版年, 出版付き, 番号, 見出し), 類似度)] -> この形になっている
    """
    
    # speech_text = '物価下がらなかったのね'
    speech_v = vectorize_word(speech_text)  # 発言をベクトル化

    similarity_dic = {}

    for company, year, month, day, headline, vector in headline_list:
        cs = cos_sim(speech_v, vector)  # 発言と見出しテキストのcos類似度を計算

        if cs >= 0.75:  # cos類似度が閾値以上であれば提示する見出しリストに加える
            similarity_dic[(company, year, month, day, headline)] = cs

    return sorted(similarity_dic.items(), key=lambda x: x[1], reverse=True)

In [17]:
import json

In [25]:
from PIL import Image

def make_json(sim_result):
    """類似度分析の結果をjsonファイルに書き出し

    Args:
        sim_result (List(tuple(tuple(str, str, str, str, str), float))): 類似度で降順にソートしたリスト [((会社名, 出版年, 出版付き, 番号, 見出し), 類似度)]

    Returns:
        flask.json.jsonify: json形式にparseされた類似度分析の結果
    """

    # 見出し画像を格納しているパス
    PIC_DIR_PATH = 'C:\\xampp\\htdocs\\KENBUN_landmark_Integration_nakano\\img\\landmark\\769_hl_10_trimLandmark\\scale0\\SNBN\\'

    result_list = []

    for result in sim_result:

        # if i < 10:  # 類似度が高い順に10個選出
        append_dic = {}  # 初期化

        company_name = result[0][0]  # 会社名
        year = result[0][1]  # 出版年
        month = result[0][2]  # 出版月
        day = result[0][3]  # 識別番号
        value = result[0][4].replace('\n', ' ')  # 見出しテキスト, 改行排除
        cs = result[1]  # cos類似度

        path = PIC_DIR_PATH + '\\'.join([company_name, year, month, day]) + '.jpg'
        if os.path.exists(path):
            img = Image.open(path)
            width, height = img.size  # 画像の幅と高さを取得
        else:  # エラーを避けるために0を代入
            width = height = 0

        append_dic = {"company_name": company_name,
                      "year": year,
                      "month": month,
                      "day": day,
                      "value": value,
                      "width": width,
                      "height": height
                      # "cos_sim": cs
                      }

        result_list.append(append_dic)

    # pprint.pprint(result_list) # 中身確認

    with open('test_result.json', 'w', encoding='utf-8') as f:
        json.dump(result_list, f, ensure_ascii=False, indent=4)  # 非asciiでjsonファイルに保存

In [19]:
test_text = '大分'

In [20]:
similarity_headlines = get_all_similarity(headline_list, test_text)

In [21]:
similarity_headlines

[(('大分合同新聞', '1942年', '7月', '23-a-3_18', '2にゃい\nに對策\n大分'), 1.0000000000000002),
 (('大分合同新聞', '1943年', '8月', '26-a-3_11', '什隊 大分發招'), 1.0000000000000002),
 (('大分合同新聞', '1946年', '6月', '27-a-2_01', '「生產大分,頗落'), 1.0000000000000002),
 (('大分合同新聞', '1949年', '11月', '6-a-2_08', '大分縣將棋人會'), 1.0000000000000002),
 (('大分合同新聞', '1949年', '2月', '25-a-2_01', '(サリナ大分) マオマオ'), 1.0000000000000002),
 (('大分合同新聞', '1950年', '3月', '18-a-2_05', '大分縣圍大會附'), 1.0000000000000002),
 (('大分合同新聞', '1950年', '3月', '19-y-2_06', '大分縣回售0'), 1.0000000000000002),
 (('大分合同新聞', '1950年', '3月', '15-y-2_05', '大分縣回售AO'), 1.0000000000000002),
 (('大分合同新聞', '1952年', '1月', '27-y-2_10', '大分别府对抗棋戰'), 1.0000000000000002),
 (('大分合同新聞', '1952年', '2月', '13-y-2_08', '大分别府对抗棋戰'), 1.0000000000000002),
 (('大分合同新聞', '1952年', '2月', '24-y-2-3_12', '大分别府对抗棋戰'), 1.0000000000000002),
 (('大分合同新聞', '1953年', '6月', '8-a-3_13', '大分に'), 1.0000000000000002),
 (('大分合同新聞', '1954年', '5月', '17-a-3_08', '大分\n6A='), 1.0000000000000002),
 (('大分合同新聞', '1956年', '10月'

In [26]:
make_json(similarity_headlines)